In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

import numpy as np
import requests as rq
import io, h5py

In [2]:
data = rq.get('https://www.dropbox.com/s/c3umbo5y13sqcfp/synthetic_dataset.h5?raw=true')
data.raise_for_status()

with h5py.File(io.BytesIO(data.content), 'r') as dataset:
    x_train = np.array(dataset['X_train']).astype(np.float32).transpose([0, 2, 1])
    y_train = np.array(dataset['Y_train']).astype(np.float32)
    x_valid = np.array(dataset['X_valid']).astype(np.float32).transpose([0, 2, 1])
    y_valid = np.array(dataset['Y_valid']).astype(np.int32)
    x_test = np.array(dataset['X_test']).astype(np.float32).transpose([0, 2, 1])
    y_test = np.array(dataset['Y_test']).astype(np.int32)

print(x_train.shape, y_train.shape)

(21000, 200, 4) (21000, 12)


In [25]:
def generate_model():
  input_layer = layers.Input(shape=(200, 4))

  conv1 = layers.Conv1D(filters=64, kernel_size=13, padding='same', use_bias='false')(input_layer)
  BN = layers.BatchNormalization()(conv1)
  act = layers.Activation('relu')(BN)
  pool1 = layers.MaxPool1D(pool_size=6)(act)
  drop1 = layers.Dropout(0.05)(pool1)

  positions = tf.range(drop1.shape[1])
  context = layers.Embedding(input_dim=drop1.shape[1], output_dim=drop1.shape[2])(positions)

  contextual_meaning = tf.add(drop1, context)

  query = layers.Dense(100, name='query')(contextual_meaning)
  value = layers.Dense(100)(contextual_meaning)

  attention, weights = layers.MultiHeadAttention(num_heads=8, key_dim=32, dropout=0.55)(query, value, return_attention_scores=True)

  flat = layers.Flatten()(attention)
  fcl = layers.Dense(512, activation='relu')(flat)
  drop2 = layers.Dropout(0.5)(fcl)

  output = layers.Dense(12, activation='sigmoid')(drop2)

  model = Model(inputs=input_layer, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(curve='ROC', name='auroc'), tf.keras.metrics.AUC(curve='PR', name='aupr')])
  return model

In [26]:
model = generate_model()
model.fit(x_train, y_train, epochs=60, validation_data=(x_valid, y_valid))

Epoch 1/60
657/657 [==============================] - 7s 8ms/step - loss: 0.4167 - auroc: 0.5769 - aupr: 0.2293 - val_loss: 0.3006 - val_auroc: 0.8078 - val_aupr: 0.5572
Epoch 2/60
657/657 [==============================] - 5s 7ms/step - loss: 0.3249 - auroc: 0.7807 - aupr: 0.5174 - val_loss: 0.2663 - val_auroc: 0.8645 - val_aupr: 0.6504
Epoch 3/60
657/657 [==============================] - 5s 8ms/step - loss: 0.2916 - auroc: 0.8358 - aupr: 0.6057 - val_loss: 0.2455 - val_auroc: 0.8904 - val_aupr: 0.6996
Epoch 4/60
657/657 [==============================] - 5s 8ms/step - loss: 0.2747 - auroc: 0.8595 - aupr: 0.6482 - val_loss: 0.2260 - val_auroc: 0.9094 - val_aupr: 0.7477
Epoch 5/60
657/657 [==============================] - 5s 7ms/step - loss: 0.2591 - auroc: 0.8825 - aupr: 0.6924 - val_loss: 0.2094 - val_auroc: 0.9296 - val_aupr: 0.7871
Epoch 6/60
657/657 [==============================] - 5s 7ms/step - loss: 0.2377 - auroc: 0.9041 - aupr: 0.7371 - val_loss: 0.1841 - val_auroc: 0.9447